In [ ]:
# pip install transformers==3

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
#config
import transformers

MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 8
EPOCHS = 4
BASE_MODEL_PATH = r"C:\Users\Prakash\Desktop\bert\bert uncased"
MODEL_PATH = r"C:\Users\Prakash\Desktop\bert\model.bin"
TRAINING_FILE = r"C:\Users\Prakash\Desktop\bert\ner_dataset.csv"
TOKENIZER = transformers.BertTokenizer.from_pretrained(
    BASE_MODEL_PATH,
    do_lower_case=True
)

In [ ]:
# pip install torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio===0.7. 0 -f https://download.pytorch.org/whl/torch_stable.html
# !conda install -c pytorch torchvision cudatoolkit=10.1 pytorch

In [2]:
#dataset
import torch

class EntityDataset:
    def __init__(self, texts, pos, tags):
     
        self.texts = texts
        self.pos = pos
        self.tags = tags
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = self.texts[item]
        pos = self.pos[item]
        tags = self.tags[item]

        ids = []
        target_pos = []
        target_tag =[]

        for i, s in enumerate(text):
            inputs = TOKENIZER.encode(
                s,
                add_special_tokens=False
            )
            
            input_len = len(inputs)
            ids.extend(inputs)
            target_pos.extend([pos[i]] * input_len)
            target_tag.extend([tags[i]] * input_len)

        ids = ids[:MAX_LEN - 2]
        target_pos = target_pos[:MAX_LEN - 2]
        target_tag = target_tag[:MAX_LEN - 2]

        ids = [101] + ids + [102]
        target_pos = [0] + target_pos + [0]
        target_tag = [0] + target_tag + [0]

        mask = [1] * len(ids)
        token_type_ids = [0] * len(ids)

        padding_len = MAX_LEN - len(ids)

        ids = ids + ([0] * padding_len)
        mask = mask + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        target_pos = target_pos + ([0] * padding_len)
        target_tag = target_tag + ([0] * padding_len)

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "target_pos": torch.tensor(target_pos, dtype=torch.long),
            "target_tag": torch.tensor(target_tag, dtype=torch.long),
        }

In [ ]:
# #engine
# from tqdm import tqdm


# def train_fn(data_loader, model, optimizer, device, scheduler):
#     model.train()
#     final_loss = 0
#     for data in tqdm(data_loader, total=len(data_loader)):
#         for k, v in data.items():
#             data[k] = v.to(device)
#         optimizer.zero_grad()
#         _, _, loss = model(**data)
#         loss.backward()
#         optimizer.step()
#         scheduler.step()
#         final_loss += loss.item()
#     return final_loss / len(data_loader)


# def eval_fn(data_loader, model, device):
#     model.eval()
#     final_loss = 0
#     for data in tqdm(data_loader, total=len(data_loader)):
#         for k, v in data.items():
#             data[k] = v.to(device)
#         _, _, loss = model(**data)
#         final_loss += loss.item()
#     return final_loss / len(data_loader)

In [3]:
#model
import torch.nn as nn
from transformers.modeling_bert import BertModel
def loss_fn(output, target, mask, num_labels):
    lfn = nn.CrossEntropyLoss()
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(lfn.ignore_index).type_as(target)
    )
    loss = lfn(active_logits, active_labels)
    return loss


class EntityModel(nn.Module):
    def __init__(self, num_tag, num_pos):
        super(EntityModel, self).__init__()
        self.num_tag = num_tag
        self.num_pos = num_pos
        self.bert = BertModel.from_pretrained(BASE_MODEL_PATH)
        self.bert_drop_1 = nn.Dropout(0.3)
        self.bert_drop_2 = nn.Dropout(0.3)
        self.out_tag = nn.Linear(768, self.num_tag)
        self.out_pos = nn.Linear(768, self.num_pos)
    
    def forward(self, ids, mask, token_type_ids, target_pos, target_tag):
        o1, _ = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)

        bo_tag = self.bert_drop_1(o1)
        bo_pos = self.bert_drop_2(o1)

        tag = self.out_tag(bo_tag)
        pos = self.out_pos(bo_pos)

        loss_tag = loss_fn(tag, target_tag, mask, self.num_tag)
        loss_pos = loss_fn(pos, target_pos, mask, self.num_pos)

        loss = (loss_tag + loss_pos) / 2

        return tag, pos, loss

In [ ]:
# #train
# import pandas as pd
# import numpy as np

# import joblib


# from sklearn import preprocessing
# from sklearn import model_selection

# from transformers import AdamW
# from transformers import get_linear_schedule_with_warmup




# def process_data(data_path):
#     df = pd.read_csv(data_path, encoding="latin-1")
#     df.loc[:, "Sentence #"] = df["Sentence #"].fillna(method="ffill")

#     enc_pos = preprocessing.LabelEncoder()
#     enc_tag = preprocessing.LabelEncoder()

#     df.loc[:, "POS"] = enc_pos.fit_transform(df["POS"])
#     df.loc[:, "Tag"] = enc_tag.fit_transform(df["Tag"])

#     sentences = df.groupby("Sentence #")["Word"].apply(list).values
#     pos = df.groupby("Sentence #")["POS"].apply(list).values
#     tag = df.groupby("Sentence #")["Tag"].apply(list).values
#     return sentences, pos, tag, enc_pos, enc_tag


# if __name__ == "__main__":
#     sentences, pos, tag, enc_pos, enc_tag = process_data(TRAINING_FILE)
    
#     meta_data = {
#         "enc_pos": enc_pos,
#         "enc_tag": enc_tag
#     }

#     joblib.dump(meta_data, "/content/drive/My Drive/bert/meta.bin")

#     num_pos = len(list(enc_pos.classes_))
#     num_tag = len(list(enc_tag.classes_))

#     (
#         train_sentences,
#         test_sentences,
#         train_pos,
#         test_pos,
#         train_tag,
#         test_tag
#     ) = model_selection.train_test_split(sentences, pos, tag, random_state=42, test_size=0.1)

#     train_dataset = EntityDataset(
#         texts=train_sentences, pos=train_pos, tags=train_tag
#     )

#     train_data_loader = torch.utils.data.DataLoader(
#         train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4
#     )

#     valid_dataset = EntityDataset(
#         texts=test_sentences, pos=test_pos, tags=test_tag
#     )

#     valid_data_loader = torch.utils.data.DataLoader(
#         valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1
#     )

#     device = torch.device("cuda")
#     model = EntityModel(num_tag=num_tag, num_pos=num_pos)
#     model.to(device)

#     param_optimizer = list(model.named_parameters())
#     no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
#     optimizer_parameters = [
#         {
#             "params": [
#                 p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
#             ],
#             "weight_decay": 0.001,
#         },
#         {
#             "params": [
#                 p for n, p in param_optimizer if any(nd in n for nd in no_decay)
#             ],
#             "weight_decay": 0.0,
#         },
#     ]

#     num_train_steps = int(len(train_sentences) / TRAIN_BATCH_SIZE * EPOCHS)
#     optimizer = AdamW(optimizer_parameters, lr=3e-5)
#     scheduler = get_linear_schedule_with_warmup(
#         optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
#     )

#     best_loss = np.inf
#     for epoch in range(EPOCHS):
#         train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
#         test_loss = eval_fn(valid_data_loader, model, device)
#         print(f"Train Loss = {train_loss} Valid Loss = {test_loss}")
#         if test_loss < best_loss:
#             torch.save(model.state_dict(), MODEL_PATH)
#             best_loss = test_loss

In [15]:
#predict

import joblib
if __name__ == "__main__":
  
    
    meta_data = joblib.load(r"C:\Users\Prakash\Desktop\bert\meta.bin")
    enc_pos = meta_data["enc_pos"]
    enc_tag = meta_data["enc_tag"]

    num_pos = len(list(enc_pos.classes_))
    num_tag = len(list(enc_tag.classes_))

    sentence = "I will be going to chennai today to attend microsoft event"
    sentence_org = "I will be going to chennai today to attend microsoft event"
    
    
    
    tokenized_sentence = TOKENIZER.encode(sentence)

    sentence = sentence.split()
    print(sentence)
    print(tokenized_sentence)

    test_dataset = EntityDataset(
        texts=[sentence], 
        pos=[[0] * len(sentence)], 
        tags=[[0] * len(sentence)]
    )

    device = torch.device("cpu")
    model = EntityModel(num_tag=num_tag, num_pos=num_pos)
    model.load_state_dict(torch.load(MODEL_PATH,map_location=device))
    model.to(device)
    

    with torch.no_grad():
        data = test_dataset[0]
        for k, v in data.items():
            data[k] = v.to(device).unsqueeze(0)
        tag, pos, _ = model(**data)

        print(
            enc_tag.inverse_transform(
                tag.argmax(2).cpu().numpy().reshape(-1)
            )[:len(tokenized_sentence)]
        )
        print(
            enc_pos.inverse_transform(
                pos.argmax(2).cpu().numpy().reshape(-1)
            )[:len(tokenized_sentence)]
        )
    pos_list = enc_pos.inverse_transform(
                pos.argmax(2).cpu().numpy().reshape(-1)
            )[:len(tokenized_sentence)][1:-1]
    fil_options = []
    for word, pos in zip(sentence,pos_list):
        print("Word :  %s     Pos : %s" %(word,pos))
        s = sentence_org
        if pos=='NNP':
            fil_options.append(word)
            s = s.replace(word,'__________')
        sentence_org = s
    print('options to fill - ',fil_options)
    print('question - ',s)

['I', 'will', 'be', 'going', 'to', 'chennai', 'today', 'to', 'attend', 'microsoft', 'event']
[101, 1045, 2097, 2022, 2183, 2000, 12249, 2651, 2000, 5463, 7513, 2724, 102]
['B-art' 'O' 'O' 'O' 'O' 'O' 'B-geo' 'B-tim' 'O' 'O' 'B-org' 'O' 'B-art']
['$' 'PRP' 'MD' 'VB' 'VBG' 'TO' 'NNP' 'NN' 'TO' 'VB' 'NNP' 'NN' '$']
Word :  I     Pos : PRP
Word :  will     Pos : MD
Word :  be     Pos : VB
Word :  going     Pos : VBG
Word :  to     Pos : TO
Word :  chennai     Pos : NNP
Word :  today     Pos : NN
Word :  to     Pos : TO
Word :  attend     Pos : VB
Word :  microsoft     Pos : NNP
Word :  event     Pos : NN
options to fill -  ['chennai', 'microsoft']
question -  I will be going to __________ today to attend __________ event
